# ML Baseline Models
Objectives:
- Establish baseline predictive models for estimating subsurface pressure behavior within the CO₂ storage reservoir.
- Train a suite of classical regression models - Linear Regression, Random Forest, and Gradient Boosting - to provide interpretable baseline performance benchmarks.
- Introduce a neural-network–based baseline (MLP) to assess the advantages of nonlinear, high-capacity function approximators for modeling complex reservoir dynamics.
- Apply consistent training and validation workflows, including data scaling, early stopping, and standardized evaluation metrics (MSE, R²).

**Source:** Society of Petroleum Engineers (SPE)  
**Dataset:** SPE Comparative Solution Project - Model 11C (3D CO₂ Injection)

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import joblib

In [ ]:
# Load data
PROCESSED = Path(r"C:\Users\tetec\Documents\Data Project Coding\.vscode\Project data\spe11c\data\processed")

X_train_scaled = np.load(PROCESSED / 'X_train_scaled.npy')
X_test_scaled = np.load(PROCESSED / 'X_test_scaled.npy')
y_train = np.load(PROCESSED / 'y_train.npy')
y_test = np.load(PROCESSED / 'y_test.npy')

print(X_train_scaled.shape, y_train.shape)


(3200, 27) (3200,)


## 1. Train Classical Regression Models

In [ ]:
# basic models
models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
}

In [ ]:
models_dir = Path(r"C:\Users\tetec\Documents\Data Project Coding\.vscode\Project data\spe11c\models")
models_dir.mkdir(parents=True, exist_ok=True)

# Train and Evaluate
results = []

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train_scaled, y_train)
    
    y_pred = model.predict(X_test_scaled)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"{name} - MSE: {mse:.4f}, R2: {r2:.4f}\n")
    
    # save model
    joblib.dump(model, rf'C:\Users\tetec\Documents\Data Project Coding\.vscode\Project data\spe11c\models\{name}_baseline.pkl')
    
    results.append({'model': name, 'MSE': mse, 'R2': r2})

# Summary dataframe
results_df = pd.DataFrame(results)
results_df

Training LinearRegression...
LinearRegression - MSE: 14.8650, R2: 1.0000

Training RandomForest...
RandomForest - MSE: 357547962.5000, R2: 1.0000

Training GradientBoosting...
GradientBoosting - MSE: 529043140.6517, R2: 1.0000



,model,MSE,R2
0,LinearRegression,1.486500e+01,1.000000
1,RandomForest,3.575480e+08,0.999966
2,GradientBoosting,5.290431e+08,0.999950


## 2. Neural Network Baseline model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1,1)).flatten()
y_test_scaled = y_scaler.transform(y_test.reshape(-1,1)).flatten()

# def mlp
def build_mlp(input_dim):
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='linear')
    ])
    return model

mlp = build_mlp(X_train_scaled.shape[1])

mlp.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='mse',
    metrics=['mse']
)

# Early stopping
es = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Train
history = mlp.fit(
    X_train_scaled,
    y_train_scaled,
    validation_split=0.2,
    epochs=300,
    batch_size=32,
    callbacks=[es],
    verbose=1
)

# Predict (unscale y)
y_pred_scaled = mlp.predict(X_test_scaled).flatten()
y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1,1)).flatten()

# Evaluate
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"NN MSE: {mse:.4f}")
print(f"NN R2: {r2:.4f}")

Epoch 1/300
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0654 - mse: 0.0654 - val_loss: 9.9326e-04 - val_mse: 9.9326e-04
Epoch 2/300
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.4532e-04 - mse: 7.4532e-04 - val_loss: 4.4517e-04 - val_mse: 4.4517e-04
Epoch 3/300
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.8675e-04 - mse: 4.8675e-04 - val_loss: 3.6905e-04 - val_mse: 3.6905e-04
Epoch 4/300
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.2756e-04 - mse: 3.2756e-04 - val_loss: 8.5476e-04 - val_mse: 8.5476e-04
Epoch 5/300
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.7517e-04 - mse: 3.7517e-04 - val_loss: 2.5801e-04 - val_mse: 2.5801e-04
Epoch 6/300
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.7578e-04 - mse: 1.7578e-04 - val_loss: 2.3833e-04 - val_mse: 2.3833e-04
Epoch 7/300
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.7205e-04 - mse: 1.7205e-04 - val_loss: 1.6311e-04 - val_mse: 1.6311e-04
Epoch 8/300
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.0188e-04 - mse: 

In [16]:
mlp.save(r"C:\Users\tetec\Documents\Data Project Coding\.vscode\Project data\spe11c\models\MLP_baseline.h5")